## Search over OV heads for pairs of nouns adjectives

In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    print("Running as a Colab notebook")
    %pip install git+https://github.com/JayBaileyCS/TransformerLens.git # TODO: Change!
    # Install Neel's personal plotting utils
    %pip install git+https://github.com/neelnanda-io/neel-plotly.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
    %pip install typing-extensions
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_111606/3645427076.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_111606/3645427076.py:23: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "png"
from transformer_lens import HookedTransformer, SVDInterpreter
import numpy as np
import torch
import pysvelte
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device = }")

device = 'cpu'


In [4]:
model = HookedTransformer.from_pretrained("gpt2-small", fold_ln=False, center_writing_weights=False, center_unembed=False)
all_tokens = [model.to_str_tokens(np.array([i])) for i in range(model.cfg.d_vocab)]
all_tokens = [all_tokens[i][0] for i in range(model.cfg.d_vocab)]

# Utility function to plot values in the same style as the Conjecture post.
def plot_matrix(matrix, tokens, k=10, filter="topk"):
  return pysvelte.TopKTable(tokens=all_tokens, activations=matrix, obj_type="SVD direction", k=k, filter=filter)

svd_interpreter = SVDInterpreter(model)


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


### Get the heatmaps and store them in files


In [17]:
for i in range(12):
    for j in range(12):
        with open(f"OV_projections/Layer{i}Head{j}.html","w") as f:
            ov = svd_interpreter.get_singular_vectors('OV', layer_index=i,head_index=j)
            a=plot_matrix(ov, all_tokens)
            b=a.html_page_str()
            f.write(b)

/home/gerard/.local/lib/python3.10/site-packages/pysvelte/svelte/src/TopKTable.py:59: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



### Get the heatmaps and store them in files


In [18]:
for i in range(12):
    with open(f"W_in_projections/Layer{i}.html","w") as f:
        w_in = svd_interpreter.get_singular_vectors('w_in', layer_index=i)
        a=plot_matrix(w_in, all_tokens)
        b=a.html_page_str()
        f.write(b)

/home/gerard/.local/lib/python3.10/site-packages/pysvelte/svelte/src/TopKTable.py:59: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



### Get the heatmaps and store them in files


In [19]:
for i in range(12):
    with open(f"W_out_projections/Layer{i}.html","w") as f:
        w_out = svd_interpreter.get_singular_vectors('w_out', layer_index=i)
        a=plot_matrix(w_out, all_tokens)
        b=a.html_page_str()
        f.write(b)